In [ ]:
!pip install Flask flask-ngrok torch torchvision

In [1]:
import os
from flask import Flask, request, jsonify, render_template_string
from flask_ngrok import run_with_ngrok
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image

In [2]:
# Model definition (same as in your model.py)
class BrainTumorClassifier(nn.Module):
    def __init__(self, num_classes):
        super(BrainTumorClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [3]:
# Load the model
model_path = 'model_new_diseases.pth'
num_classes = 4  # Adjust this based on your number of classes

model = BrainTumorClassifier(num_classes=num_classes)
model.load_state_dict(torch.load(model_path))
model.eval()

BrainTumorClassifier(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=50176, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=4, bias=True)
)

In [4]:
# Define the transform
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [5]:
# Define class names
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']  # Replace with your actual class names

def predict_brain_tumor(image_path, model, transform, class_names):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
        class_index = predicted.item()
        class_label = class_names[class_index]
        confidence = torch.nn.functional.softmax(output, dim=1)[0][class_index].item()
    return class_label, confidence

In [6]:
# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)

In [7]:
@app.route('/')
def index():
    return render_template_string('''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Brain Tumor Classification</title>
        <style>
            body {
                background-color: #57BD9E;
                color: black;
                display: flex;
                flex-direction: column;
                align-items: center;
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 20px;
            }
            h1 {
                text-align: center;
                width: 100%;
                margin-bottom: 20px;
                font-size: 2.5em;
                font-weight: bold;
                color: white;
                background-color: #333;
                padding: 10px;
                border-radius: 10px;
            }
            #form-container {
                display: flex;
                flex-direction: column;
                align-items: flex-start;
                width: 100%;
                max-width: 400px;
            }
            form {
                display: flex;
                flex-direction: column;
                align-items: flex-start;
                width: 100%;
            }
            input[type="file"] {
                margin-bottom: 20px;
            }
            #uploaded-image {
                max-width: 100%;
                margin-bottom: 20px;
            }
            input[type="submit"] {
                margin-bottom: 20px;
            }
            #result {
                font-size: 1.2em;
            }
        </style>
    </head>
    <body>
        <h1>Brain Tumor Classification</h1>
        <div id="form-container">
            <form action="/predict" method="post" enctype="multipart/form-data">
                <input type="file" name="file" id="file-input">
                <img id="uploaded-image" src="" alt="Uploaded Image" style="display: none;">
                <input type="submit" value="Predict">
            </form>
            <div id="result"></div>
        </div>
        <script>
            const fileInput = document.getElementById('file-input');
            const uploadedImage = document.getElementById('uploaded-image');

            fileInput.addEventListener('change', () => {
                const file = fileInput.files[0];
                if (file) {
                    const reader = new FileReader();
                    reader.onload = (e) => {
                        uploadedImage.src = e.target.result;
                        uploadedImage.style.display = 'block';
                    };
                    reader.readAsDataURL(file);
                }
            });

            const form = document.querySelector('form');
            form.addEventListener('submit', async (e) => {
                e.preventDefault();
                const formData = new FormData(form);
                const response = await fetch('/predict', {
                    method: 'POST',
                    body: formData
                });
                const result = await response.json();
                document.getElementById('result').innerText = `Class: ${result.class}, Confidence: ${result.confidence.toFixed(4)}`;
            });
        </script>
    </body>
    </html>
    ''')


In [ ]:
@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'})

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'})

    if file:
        file_path = os.path.join('uploads', file.filename)
        file.save(file_path)
        class_label, confidence = predict_brain_tumor(file_path, model, transform, class_names)
        return jsonify({'class': class_label, 'confidence': confidence})

if __name__ == '__main__':
    if not os.path.exists('uploads'):
        os.makedirs('uploads')
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Users\Aniketkadam\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Aniketkadam\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "C:\Users\Aniketkadam\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Aniketkadam\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\connectionpool.py"